# 0. Importing packages

In [1]:
# Load the autoreload extension to automatically reload modules before executing code (to avoid restarting the kernel)
%load_ext autoreload 
# NB. uncomment the line above first time you run this cell
%autoreload 2

In [2]:
import json
import pandas as pd
import numpy as np

# Custom functions
import resources.filter_functions as filter

## 0.1 File Paths

In [3]:
# File paths
import resources.filepaths as fp

fp_main = fp.fp_main
fp_main_output = fp.fp_main_output

# To output data that has to go to external s-drive
fp_main_external = fp.fp_main_external
fp_output_external = fp.fp_output_external

# 1. Importing data

*Loading in first and second tier data*

In [ ]:
# Opening the files
first_tier_file_name = "first_tier_ties_extended.parquet.gzip"
second_tier_file_name = "second_tier_userinfo.jsonl"
fp_first_tier = fp_output_external / first_tier_file_name
fp_second_tier = fp_output_external / second_tier_file_name

# Load the first tier data
first_tier_data_clean = pd.read_parquet(fp_first_tier)

# Columns that should be lists
columns_to_convert = ["inferred_company"]

for column in columns_to_convert:
    if column in first_tier_data_clean.columns:
        first_tier_data_clean[column] = first_tier_data_clean[column].apply(
            lambda x: list(x) if isinstance(x, (list, np.ndarray)) else x
        )

# Load the second tier data
with open(fp_second_tier, "r") as f:
    second_tier_list = [json.loads(line) for line in f]

# Type hint so Pylance can infer the type of the variable
second_tier_data: pd.DataFrame = pd.DataFrame(second_tier_list)


# 2. Filter second-tier users on company

In [ ]:
# Make a copy
second_tier_users_clean = second_tier_data.copy()

# Find matched company strings
second_tier_users_clean["matched_company_strings"] = second_tier_users_clean.apply(
    lambda user_row: filter.search_for_company(
        [
            v
            for v in [
                user_row.get("user_login"),
                user_row.get("listed_company"),
                user_row.get("email"),
                user_row.get("bio"),
                user_row.get("blog"),
            ]
            if v and str(v).strip()
        ]
    ),
    axis=1,
)  # type: ignore

# Get the keys of the element in the dictionary of matched company strings
second_tier_users_clean["inferred_company"] = second_tier_users_clean[
    "matched_company_strings"
].apply(lambda x: list(x.keys()) if isinstance(x, dict) else [])

# Filter on rows that have an inferred company
second_tier_data_clean = second_tier_users_clean[
    second_tier_users_clean["inferred_company"].apply(len) > 0
]

# 3. Merging first-tier and second-tier

In [ ]:
# Assign tier labels to each dataset
first_tier_data_clean["tier"] = 1
second_tier_data_clean["tier"] = 2

# Combine both tiers into a single dataframe
merged_users = pd.concat(
    [first_tier_data_clean, second_tier_data_clean], ignore_index=True
)
print(f"[INFO] Merged user data shape: {merged_users.shape}")

# Remove duplicate users based on GitHub login
unique_users_data = merged_users.drop_duplicates(subset="user_login").reset_index(
    drop=True
)
print(f"[INFO] Deduplicated user data shape: {unique_users_data.shape}")

[INFO] Merged user data shape: (202, 22)
[INFO] Deduplicated user data shape: (194, 22)


/var/folders/sw/l43jl9d15098xcnblnvk66jc0000gn/T/ipykernel_68873/3963180515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_tier_data_clean['tier'] = 2


# 4. Resolve multiple matches

In [ ]:
# Access output path
output_path = "../output/resolved_multicompany_cases.jsonl"

# Resolve multiple companies
unique_users_data_clean = filter.resolve_multiple_companies(
    unique_users_data,
    output_path=output_path,
)

[mbetrifork] has multiple company matches:
Inferred Companies: ['trifork', 'must']

Bio Information:
  user_login: mbetrifork
  search_with_company: must
  usertype: User
  listed_company: Trifork Public A/S
  email: mbe@trifork.com
  bio: Family, religion, friendship. These are the three demons you must slay if you wish to succeed in business.
  blog: 


# 5. Output file

In [ ]:
# Output the filtered users
unique_users_data_clean.to_parquet(fp_output_external / "final_dataset.gzip.parquet")